In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Assuming your model is saved in a file named 'my_model.h5'
model_path = "/kaggle/input/uni-rec-latest-model/tensorflow2/uni_rec_latest/1/nn_for_uni_rec_wt_toefl_latest.h5"

# Load the model
model = load_model(model_path)

2024-05-09 15:38:43.732938: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 15:38:43.733034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 15:38:43.859584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
import pandas as pd

path="/kaggle/input/final-dataset/final dataset using knn.csv"
df=pd.read_csv(path)
df.head()

,Unnamed: 0,University Name,toeflScore,Course Name,GRE_AWA,GRE_QUANT,GRE_VERBAL,GPA,Degree,Publications,Research Experience,Work experience,min_gpa,gre_total,min_gre
0,2,university of illinois urbana champaign,80,accounting,3.5,36,27,3.37,Masters,0,0,0,3.0,63,260.0
1,10,pennsylvania state university,80,acoustics,3.5,38,33,3.60,PhD,0,0,1,2.0,71,280.0
2,11,pennsylvania state university,80,acoustics,4.0,39,26,3.67,PhD,0,0,0,2.0,65,280.0
3,15,boston university,80,actuarial science,3.0,24,37,3.67,Masters,0,0,0,3.0,61,290.0
4,16,boston university,80,actuarial science,2.0,26,27,2.90,Masters,0,0,0,3.0,53,290.0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34196 entries, 0 to 34195
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           34196 non-null  int64  
 1   University Name      34196 non-null  object 
 2   toeflScore           34196 non-null  int64  
 3   Course Name          34196 non-null  object 
 4   GRE_AWA              34196 non-null  float64
 5   GRE_QUANT            34196 non-null  int64  
 6   GRE_VERBAL           34196 non-null  int64  
 7   GPA                  34196 non-null  float64
 8   Degree               34196 non-null  object 
 9   Publications         34196 non-null  int64  
 10  Research Experience  34196 non-null  int64  
 11  Work experience      34196 non-null  int64  
 12  min_gpa              34196 non-null  float64
 13  gre_total            34196 non-null  int64  
 14  min_gre              34196 non-null  float64
dtypes: float64(4), int64(8), object(3)
m

In [14]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [15]:
numerical_columns = ['toeflScore', 'GRE_AWA', 'GRE_QUANT', 'GRE_VERBAL', 'GPA','min_gpa','gre_total']

for col in numerical_columns:
  if col=='min_gpa' or col=='GPA' or col=='GRE_AWA':
    df[col]=df[col].astype(float).round(2)
  else:
    df[col]=df[col].astype(int)

df.head()

,University Name,toeflScore,Course Name,GRE_AWA,GRE_QUANT,GRE_VERBAL,GPA,Degree,Publications,Research Experience,Work experience,min_gpa,gre_total,min_gre
0,university of illinois urbana champaign,80,accounting,3.5,36,27,3.37,Masters,0,0,0,3.0,63,260.0
1,pennsylvania state university,80,acoustics,3.5,38,33,3.60,PhD,0,0,1,2.0,71,280.0
2,pennsylvania state university,80,acoustics,4.0,39,26,3.67,PhD,0,0,0,2.0,65,280.0
3,boston university,80,actuarial science,3.0,24,37,3.67,Masters,0,0,0,3.0,61,290.0
4,boston university,80,actuarial science,2.0,26,27,2.90,Masters,0,0,0,3.0,53,290.0


In [16]:
df = df.groupby('Course Name').filter(lambda x: len(x) >= 50)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30888 entries, 14 to 34174
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   University Name      30888 non-null  object 
 1   toeflScore           30888 non-null  int64  
 2   Course Name          30888 non-null  object 
 3   GRE_AWA              30888 non-null  float64
 4   GRE_QUANT            30888 non-null  int64  
 5   GRE_VERBAL           30888 non-null  int64  
 6   GPA                  30888 non-null  float64
 7   Degree               30888 non-null  object 
 8   Publications         30888 non-null  int64  
 9   Research Experience  30888 non-null  int64  
 10  Work experience      30888 non-null  int64  
 11  min_gpa              30888 non-null  float64
 12  gre_total            30888 non-null  int64  
 13  min_gre              30888 non-null  float64
dtypes: float64(4), int64(7), object(3)
memory usage: 3.5+ MB


In [18]:

profile_df = pd.read_csv("/kaggle/input/validation-datasetpreprocessed/Validation Data Preprocessed (1).csv")



In [19]:
profile_df

,Unnamed: 0,Name,Degree,GPA,GPA Scale,GRE V,GRE Q,GRE AWA,TOEFL,Budget,Research,work exp,publications,Course Name,Admitted University Name,Recommended Universities
0,0,Mohana Krishna,Masters,7.00,10,141,163,5.0,94,50000,0,1,0,22,87,NaN
1,1,Pranay Deep Reddy,Masters,6.55,10,155,170,3.0,95,40000,1,1,1,3,17,NaN
2,2,Kaustubh Rai,Masters,8.67,10,149,165,3.0,106,60000,1,1,1,3,110,NaN
3,4,Apoorva Kulkarni,Masters,9.37,10,152,166,4.0,109,500000,1,1,1,3,104,NaN
4,5,Apoorva Kulkarni,Masters,9.37,10,152,166,4.0,109,500000,1,1,1,3,77,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,212,Malvika,Masters,7.95,10,157,170,4.5,117,100000,0,1,0,55,103,NaN
177,213,Malvika,Masters,7.95,10,157,170,4.5,117,100000,0,1,0,55,49,NaN
178,214,Malvika,Masters,7.95,10,157,170,4.5,117,100000,0,1,0,55,94,NaN
179,215,Vignesh Sitaraman,Masters,7.03,10,165,167,4.0,111,100000,0,0,0,11,50,NaN


In [20]:
profile_df.drop(['Unnamed: 0'],axis=1,inplace=True)


In [21]:
profile_df_copy=profile_df.copy()

In [22]:
profile_df_copy

,Name,Degree,GPA,GPA Scale,GRE V,GRE Q,GRE AWA,TOEFL,Budget,Research,work exp,publications,Course Name,Admitted University Name,Recommended Universities
0,Mohana Krishna,Masters,7.00,10,141,163,5.0,94,50000,0,1,0,22,87,NaN
1,Pranay Deep Reddy,Masters,6.55,10,155,170,3.0,95,40000,1,1,1,3,17,NaN
2,Kaustubh Rai,Masters,8.67,10,149,165,3.0,106,60000,1,1,1,3,110,NaN
3,Apoorva Kulkarni,Masters,9.37,10,152,166,4.0,109,500000,1,1,1,3,104,NaN
4,Apoorva Kulkarni,Masters,9.37,10,152,166,4.0,109,500000,1,1,1,3,77,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Malvika,Masters,7.95,10,157,170,4.5,117,100000,0,1,0,55,103,NaN
177,Malvika,Masters,7.95,10,157,170,4.5,117,100000,0,1,0,55,49,NaN
178,Malvika,Masters,7.95,10,157,170,4.5,117,100000,0,1,0,55,94,NaN
179,Vignesh Sitaraman,Masters,7.03,10,165,167,4.0,111,100000,0,0,0,11,50,NaN


In [23]:
columns_to_drop = ['Name', 'Budget', 'GPA Scale','Course Name','Admitted University Name','Recommended Universities']  # Replace ... with your column names
profile_df = profile_df.drop(columns=columns_to_drop)

In [24]:
profile_df

,Degree,GPA,GRE V,GRE Q,GRE AWA,TOEFL,Research,work exp,publications
0,Masters,7.00,141,163,5.0,94,0,1,0
1,Masters,6.55,155,170,3.0,95,1,1,1
2,Masters,8.67,149,165,3.0,106,1,1,1
3,Masters,9.37,152,166,4.0,109,1,1,1
4,Masters,9.37,152,166,4.0,109,1,1,1
...,...,...,...,...,...,...,...,...,...
176,Masters,7.95,157,170,4.5,117,0,1,0
177,Masters,7.95,157,170,4.5,117,0,1,0
178,Masters,7.95,157,170,4.5,117,0,1,0
179,Masters,7.03,165,167,4.0,111,0,0,0


In [25]:
# df.head()
new_column_names = ['Degree', 'GPA', 'GRE_VERBAL','GRE_QUANT','GRE_AWA','toeflScore','Research Experience','Work experience','Publications']  # Replace ... with your column names
profile_df.columns = new_column_names

profile_df

,Degree,GPA,GRE_VERBAL,GRE_QUANT,GRE_AWA,toeflScore,Research Experience,Work experience,Publications
0,Masters,7.00,141,163,5.0,94,0,1,0
1,Masters,6.55,155,170,3.0,95,1,1,1
2,Masters,8.67,149,165,3.0,106,1,1,1
3,Masters,9.37,152,166,4.0,109,1,1,1
4,Masters,9.37,152,166,4.0,109,1,1,1
...,...,...,...,...,...,...,...,...,...
176,Masters,7.95,157,170,4.5,117,0,1,0
177,Masters,7.95,157,170,4.5,117,0,1,0
178,Masters,7.95,157,170,4.5,117,0,1,0
179,Masters,7.03,165,167,4.0,111,0,0,0


In [26]:
profile_df['Degree'] = profile_df['Degree'].replace('Masters', '1')

In [27]:
profile_df

,Degree,GPA,GRE_VERBAL,GRE_QUANT,GRE_AWA,toeflScore,Research Experience,Work experience,Publications
0,1,7.00,141,163,5.0,94,0,1,0
1,1,6.55,155,170,3.0,95,1,1,1
2,1,8.67,149,165,3.0,106,1,1,1
3,1,9.37,152,166,4.0,109,1,1,1
4,1,9.37,152,166,4.0,109,1,1,1
...,...,...,...,...,...,...,...,...,...
176,1,7.95,157,170,4.5,117,0,1,0
177,1,7.95,157,170,4.5,117,0,1,0
178,1,7.95,157,170,4.5,117,0,1,0
179,1,7.03,165,167,4.0,111,0,0,0


In [28]:
# GRE
profile_df['GRE_QUANT'] = profile_df['GRE_QUANT'] - 130
profile_df['GRE_VERBAL'] = profile_df['GRE_VERBAL'] - 130

In [29]:
#Convert GPA to scale of 4
import pandas as pd

# GPA conversion function
def convert_gpa(gpa, scale):
    if scale == 10:
        return gpa / 2.5
    elif scale ==5:
        return (gpa/5)*4
    elif scale == 20:
        return gpa / 5.0
    elif scale == 100:
        return gpa / 25.0
    else:
        return gpa  

# Convert GPA column to a 4.0 scale
profile_df['GPA'] = profile_df.apply(lambda row: convert_gpa(row['GPA'], 10), axis=1)  # Assuming GPA is on a 10-point scale

# Display the updated DataFrame
print(profile_df)


    Degree    GPA  GRE_VERBAL  GRE_QUANT  GRE_AWA  toeflScore  \
0        1  2.800          11         33      5.0          94   
1        1  2.620          25         40      3.0          95   
2        1  3.468          19         35      3.0         106   
3        1  3.748          22         36      4.0         109   
4        1  3.748          22         36      4.0         109   
..     ...    ...         ...        ...      ...         ...   
176      1  3.180          27         40      4.5         117   
177      1  3.180          27         40      4.5         117   
178      1  3.180          27         40      4.5         117   
179      1  2.812          35         37      4.0         111   
180      1  2.812          35         37      4.0         111   

     Research Experience  Work experience  Publications  
0                      0                1             0  
1                      1                1             1  
2                      1                1    

In [30]:
df = pd.concat([df, profile_df], ignore_index=True)


In [31]:
df

,University Name,toeflScore,Course Name,GRE_AWA,GRE_QUANT,GRE_VERBAL,GPA,Degree,Publications,Research Experience,Work experience,min_gpa,gre_total,min_gre
0,university of toronto,67,aeronautical engineering,4.0,40,27,3.980,Masters,1,0,0,3.00,67.0,260.0
1,university of north carolina at charlotte,70,aeronautical engineering,3.0,31,14,2.160,Masters,0,0,0,2.00,45.0,290.0
2,university of houston,71,aeronautical engineering,3.5,25,24,3.560,Masters,0,0,0,2.25,49.0,305.0
3,university of colorado boulder,75,aeronautical engineering,4.5,24,24,3.500,Masters,0,0,0,2.75,48.0,300.0
4,university of colorado boulder,75,aeronautical engineering,4.0,26,17,3.470,Masters,0,0,1,2.75,43.0,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31064,NaN,117,NaN,4.5,40,27,3.180,1,0,0,1,NaN,NaN,NaN
31065,NaN,117,NaN,4.5,40,27,3.180,1,0,0,1,NaN,NaN,NaN
31066,NaN,117,NaN,4.5,40,27,3.180,1,0,0,1,NaN,NaN,NaN
31067,NaN,111,NaN,4.0,37,35,2.812,1,0,0,0,NaN,NaN,NaN


In [32]:
df.drop(['min_gpa','gre_total','min_gre'],axis=1,inplace=True)

In [33]:
### Preprocessing the profile
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
columns=['GPA','GRE_AWA','GRE_QUANT','GRE_VERBAL','toeflScore','Publications','Research Experience','Work experience']
df[columns] = scaler.fit_transform(df[columns])

# Display the DataFrame
print(df)

                                 University Name  toeflScore  \
0                          university of toronto       -13.0   
1      university of north carolina at charlotte       -10.0   
2                          university of houston        -9.0   
3                 university of colorado boulder        -5.0   
4                 university of colorado boulder        -5.0   
...                                          ...         ...   
31064                                        NaN        37.0   
31065                                        NaN        37.0   
31066                                        NaN        37.0   
31067                                        NaN        31.0   
31068                                        NaN        31.0   

                    Course Name  GRE_AWA  GRE_QUANT  GRE_VERBAL       GPA  \
0      aeronautical engineering      0.0   0.777778         0.1  0.796875   
1      aeronautical engineering     -1.0  -0.222222        -1.2 -2.046875   


In [34]:
ip_df =df.tail(181)
ip_df.reset_index(drop=True, inplace=True)
ip_df

,University Name,toeflScore,Course Name,GRE_AWA,GRE_QUANT,GRE_VERBAL,GPA,Degree,Publications,Research Experience,Work experience
0,NaN,14.0,NaN,1.0,0.000000,-1.5,-1.046875,1,0.0,0.0,1.0
1,NaN,15.0,NaN,-1.0,0.777778,-0.1,-1.328125,1,1.0,1.0,1.0
2,NaN,26.0,NaN,-1.0,0.222222,-0.7,-0.003125,1,1.0,1.0,1.0
3,NaN,29.0,NaN,0.0,0.333333,-0.4,0.434375,1,1.0,1.0,1.0
4,NaN,29.0,NaN,0.0,0.333333,-0.4,0.434375,1,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
176,NaN,37.0,NaN,0.5,0.777778,0.1,-0.453125,1,0.0,0.0,1.0
177,NaN,37.0,NaN,0.5,0.777778,0.1,-0.453125,1,0.0,0.0,1.0
178,NaN,37.0,NaN,0.5,0.777778,0.1,-0.453125,1,0.0,0.0,1.0
179,NaN,31.0,NaN,0.0,0.444444,0.9,-1.028125,1,0.0,0.0,0.0


In [35]:
ip_df = ip_df.drop(columns=['University Name', 'Course Name'])

In [36]:
# ip_df = pd.concat([profile_df_copy['Course Name'], profile_df_copy['Admitted University Name']], axis=1)
ip_df = ip_df.join([profile_df_copy['Course Name']])
ip_df = ip_df.join([profile_df_copy['Admitted University Name']])                 

In [37]:
ip_df

,toeflScore,GRE_AWA,GRE_QUANT,GRE_VERBAL,GPA,Degree,Publications,Research Experience,Work experience,Course Name,Admitted University Name
0,14.0,1.0,0.000000,-1.5,-1.046875,1,0.0,0.0,1.0,22,87
1,15.0,-1.0,0.777778,-0.1,-1.328125,1,1.0,1.0,1.0,3,17
2,26.0,-1.0,0.222222,-0.7,-0.003125,1,1.0,1.0,1.0,3,110
3,29.0,0.0,0.333333,-0.4,0.434375,1,1.0,1.0,1.0,3,104
4,29.0,0.0,0.333333,-0.4,0.434375,1,1.0,1.0,1.0,3,77
...,...,...,...,...,...,...,...,...,...,...,...
176,37.0,0.5,0.777778,0.1,-0.453125,1,0.0,0.0,1.0,55,103
177,37.0,0.5,0.777778,0.1,-0.453125,1,0.0,0.0,1.0,55,49
178,37.0,0.5,0.777778,0.1,-0.453125,1,0.0,0.0,1.0,55,94
179,31.0,0.0,0.444444,0.9,-1.028125,1,0.0,0.0,0.0,11,50


In [38]:
input_df=ip_df[['GRE_AWA','GRE_QUANT','GRE_VERBAL','GPA','Course Name']]

In [39]:
logits=model.predict(input_df)
# Apply softmax to get probabilities
probabilities = tf.nn.softmax(logits)

# Convert TensorFlow tensor to numpy array
probabilities = probabilities.numpy()

# Display predicted probabilities
print(len(probabilities))

1/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step

I0000 00:00:1715269192.961339      87 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1715269192.974693      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step
181


In [57]:
import numpy as np

total_predictions=[]
for j in range(len(ip_df)):
# Extract probabilities and corresponding class labels
    probabilities = logits[j]  # Flatten the array to 1D
    class_labels = np.arange(len(probabilities))  # Assuming class indices start from 0

    # Get indices of top 10 predicted classes
    top_10_indices = np.argsort(probabilities)[::-1][:40]

    # Get corresponding probabilities and class labels for top 10
    top_10_probabilities = probabilities[top_10_indices]
    top_10_class_labels = class_labels[top_10_indices]

    # Display top 10 predicted classes along with their probabilities
    predictions=[]
    for i in range(35):
        predictions.append(top_10_class_labels[i])
        filtered_df = re_df[re_df['University Name Encoded'] == top_10_class_labels[i]]
    #     print(filtered_df.head())
        max_score=filtered_df['toeflScore'].max()*weights['toeflScore']+filtered_df['GRE_AWA'].max()*weights['GRE_AWA']+filtered_df['GRE_QUANT'].max()*weights['GRE_QUANT']+filtered_df['GRE_VERBAL'].max()*weights['GRE_VERBAL']+filtered_df['GPA'].max()*weights['GPA']
    #     print(max_score)
    #     max_score=filtered_df['Normalized Score'].max()
#         print(max_score)
        score=ip_df['GRE_AWA']*weights['GRE_AWA']+ip_df['GRE_QUANT']*weights['GRE_QUANT']+ip_df['GRE_VERBAL']*weights['GRE_VERBAL']+ip_df['GPA']*weights['GPA']
        admit_percent=((score-min_score)/(max_score-min_score))*100
#         print("Class:", top_10_class_labels[i], " Probability:", top_10_probabilities[i])
#     print("-----")
    total_predictions.append(predictions)


In [59]:
admitted_uni=ip_df['Admitted University Name'].tolist()
results=[]
for i in range(len(ip_df)):
    if(admitted_uni[i] in total_predictions[i]):
        results.append(1)
    else:
        results.append(0)
print((sum(results)/181)*100)  #validation result

60.22099447513812
